In [ ]:
# step 1: Install Ultralytics
!pip install ultralytics

In [ ]:
# This configuration is only for loading the custom model weights, since the current version (torch 2.6) causes problems.

!pip install torch==2.5
!pip install torchvision==0.20

In [3]:
# Step 2: copy your content from Drive to your home directory.

!cp -r "/content/drive/My Drive/custom model/" "/home"

In [4]:
# Step 3: replace the original files in the ultralytic directories.

!cp -r "/home/custom model/files/__init__.py" "/usr/local/lib/python3.11/dist-packages/ultralytics/nn/modules"
!cp -r "/home/custom model/files/conv.py" "/usr/local/lib/python3.11/dist-packages/ultralytics/nn/modules"
!cp -r "/home/custom model/files/tasks.py" "/usr/local/lib/python3.11/dist-packages/ultralytics/nn"
!cp -r "/home/custom model/files/loss.py" "/usr/local/lib/python3.11/dist-packages/ultralytics/utils/loss.py"

In [ ]:
# Step 4: load the custom model and the original weights from Yolov11.

from ultralytics import YOLO
model = YOLO('/home/custom model/files/yolo11n.yaml').load("yolo11n.pt")

In [ ]:
# Step 5: Training stage

import math
import os
os.environ["WANDB_MODE"] = "offline"

bz = 47 # batch size

def on_fit_epoch_end(trainer):
    # get the current epoch number from the trainer
    t = trainer.epoch + 1

    if t > 10:
      m = 1 - (math.sqrt(0.05))**1/ t * bz
      trainer.momentum = m
      print(f"Momentum: {m}")


model.add_callback("on_fit_epoch_end", on_fit_epoch_end) # increasing momentum

results = model.train(data='/home/custom model/files/data.yaml',
                      patience=25,
                      cache = 'Disk',
                      cos_lr = True,
                      single_cls= True,
                      lr0= 0.01,
                      lrf= 0.0001,
                      momentum=  0.932,
                      weight_decay=0.0005,
                      warmup_epochs= 1.22935,
                      warmup_momentum= 0.1525,
                      box= 18.27875,
                      cls= 1.32899,
                      dfl= 0.56016,
                      hsv_h= 0.0,
                      hsv_s= 0.0,
                      hsv_v= 0.0,
                      degrees= 0.0,
                      translate= 0.12431,
                      scale= 0.07643,
                      shear= 0.0,
                      perspective= 0.0,
                      flipud= 0.63,
                      fliplr= 0.63,
                      mosaic= 0.63,
                      mixup= 0.0,
                      close_mosaic=10,
                      epochs=500,
                      optimizer="SGD",
                      dropout=0.0,
                      imgsz=416,
                      batch=bz)

In [7]:
from matplotlib import image as img
from matplotlib import pyplot as plt
from PIL import Image
import cv2

def showBoundingBoxes(obj,name):

  for r in obj:

     boxes = r.boxes.xyxy
     conf = r.boxes.conf
     cls = r.boxes.cls
     im_array = r.plot()  # plot a BGR numpy array of predictions
     img = Image.fromarray(im_array[..., ::-1])  # RGB PIL image

     c = 0
     for bb in boxes:
       c = c + 1

     #plt.title("Number of Bounding Boxes:" +str(c))
     plt.axis('off')
     plt.imshow(img)
     plt.show()

In [ ]:
import os
from ultralytics import YOLO
from matplotlib import image as img
from matplotlib import pyplot as plt
from PIL import Image
import torch

test_path = "/home/custom model/dataset/test/"
test_path = os.listdir(test_path)
test_path.sort()


model = YOLO('/home/custom model/weights/best.pt')

for img in test_path:
  try:
    predict = model.predict("/home/custom model/dataset/test/" + str(img), conf= 0.5, iou=0.5)
    showBoundingBoxes(predict,img)
  except Exception as e:
    print(e)